In [35]:
import pandas as pd
import os
import geopandas as gpd
import re # regular expression
#import pyproj

In [36]:
# GLOBAL VARIABLES

# excel spreadsheet copied to data/udot folder from https://drive.google.com/file/d/1rDXm0ObugGR1zXgWUuVbzWHNt-Xs1xru/view
fnExcelAADTHistory = 'data/udot/AADTHistory.xlsx'
fnAADT2022 = 'data/udot/Traffic on Utah Highways 2022.xlsx - AADT2022.csv' # added Feb 16 2024

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/Segments_State_20231221_Draft.shp'

# filter by PLANAREA in segments shapefile
#filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

# Get AADT and Previous Forecasts from Segment Shapefile

In [37]:
# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
display(gdfSegments[gdfSegments['SEGID'].duplicated(keep=False)])
display(gdfSegments)

# show columns
#print(gdfSegments.columns.tolist())

,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,geometry


,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,geometry
0,0,0006_000.0,0.000,0.665,0.666642,UDOT,415.0,0,27,UDOT,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (236177.700 4327541.250, 237241.180..."
1,0,0006_000.7,0.665,16.022,15.369870,UDOT,415.0,0,27,UDOT,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (237241.180 4327399.720, 238150.600..."
2,0,0006_016.0,16.022,46.017,30.002021,UDOT,415.0,0,27,UDOT,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (261403.900 4327045.400, 261409.000..."
3,0,0006_046.0,46.017,60.218,14.194335,UDOT,372.0,0,27,UDOT,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (305380.160 4325741.850, 305784.500..."
4,0,0006_060.2,60.218,77.545,17.323272,UDOT,372.0,0,27,UDOT,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (324618.000 4337936.900, 324647.400..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8962,0,WFRC_8469,0.000,0.000,0.567664,WFRC,0.0,1,35,WFRC,...,1.0575,0.6429,1.1132,1.0744,1.0918,0.7585,1.0878,0.8742,0.8994,"LINESTRING (407213.374 4504030.989, 407034.934..."
8963,0,WFRC_8470,0.000,0.000,0.493148,WFRC,0.0,1,35,WFRC,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (407723.200 4503422.753, 407213.374..."
8964,0,WFRC_8471,0.000,0.000,0.405891,WFRC,0.0,1,35,WFRC,...,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370,"LINESTRING (415734.595 4481608.605, 415755.740..."
8965,0,WFRC_8472,0.000,0.000,0.253734,WFRC,0.0,1,35,WFRC,...,1.0135,0.6293,1.1063,1.0851,1.0946,0.7583,1.0925,0.8873,0.9653,"LINESTRING (424308.500 4489341.200, 424304.312..."


In [38]:
# filter segments by PLANAREA
_df = gdfSegments#[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

## get columns with AADT at beginning of name
#aadt_columns = [col for col in _df.columns if col.startswith('AADT')]
#
## get dataframe for just segids and aadt columns
#_df = _df[['SEGID'] + aadt_columns]
#
## melt to make long
#_df = _df.melt(id_vars='SEGID', var_name='YEAR', value_name='AADT')
#
## filter out zero volumes
#_df = _df[_df['AADT']>0]
#
## get YEAR from string
#_df['YEAR'] = _df['YEAR'].str[-4:].astype(int)
#
## set AADT to int
#_df['AADT'] = _df['AADT'].astype(int)
#
## set source
#_df['SOURCE'] = fnSegmentsShapefile.split('/')[-1]

dfAadtFromSegments = pd.DataFrame()#_df

dfAadtFromSegments

""


# Get Historic Data from AADTHistory UDOT Spreadsheet

In [39]:
#import excel spreadsheet
xl_file1 = pd.ExcelFile(fnExcelAADTHistory)
dfs1 = {sheet_name: xl_file1.parse(sheet_name) for sheet_name in xl_file1.sheet_names}

c:\Users\bhereth\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [40]:
#show sheet
display(xl_file1.sheet_names)
dfs1['AADT_2021']

['AADT_2021', 'AADT_Rounded', 'AADT_Unrounded']

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT_2021
0,0006PM,0.000,46.038,474
1,0006PM,46.038,77.556,424
2,0006PM,77.556,82.897,609
3,0006PM,82.897,83.911,2272
4,0006PM,83.911,87.694,3852
...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886
4569,0015PC29501,0.000,0.166,1000
4570,0015PC30554,0.000,1.135,35025
4571,0092NC00101,0.000,3.294,12201


In [41]:
# PREPARE 2021 DATA

df2021 = dfs1['AADT_2021']

df2021['YEAR'] = 2021
df2021.rename(columns={'AADT_2021':'AADT'}, inplace=True)
df2021

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT,YEAR
0,0006PM,0.000,46.038,474,2021
1,0006PM,46.038,77.556,424,2021
2,0006PM,77.556,82.897,609,2021
3,0006PM,82.897,83.911,2272,2021
4,0006PM,83.911,87.694,3852,2021
...,...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886,2021
4569,0015PC29501,0.000,0.166,1000,2021
4570,0015PC30554,0.000,1.135,35025,2021
4571,0092NC00101,0.000,3.294,12201,2021


In [42]:
# PREPARE 2022 AADT

df2022 = pd.read_csv(fnAADT2022)

df2022['YEAR'] = 2022
df2022.rename(columns={'AADT2022':'AADT','Route':'ROUTE_ID','Begin MP':'FROM_MEASURE','End MP':'TO_MEASURE','AADT_2022':'AADT'}, inplace=True)
df2022.drop(columns=['Station','Section Length','Location Description'], inplace=True)

df2022['AADT'] = df2022['AADT'].str.replace(',', '').astype(int)

df2022

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT,YEAR
0,0006PM,0.000,46.0380,441,2022
1,0006PM,46.038,77.5560,395,2022
2,0006PM,77.556,82.8970,566,2022
3,0006PM,82.897,83.9110,2113,2022
4,0006PM,83.911,87.6940,3910,2022
...,...,...,...,...,...
4553,3468PM,0.000,5.1248,2733,2022
4554,3469PM,0.000,6.9300,371,2022
4555,3470PM,0.000,1.0394,1686,2022
4556,3478PM,0.000,2.0404,965,2022


In [43]:
# GET AADTS FOR YEARS PRIOR TO 2021

# melt AADT columns to rows
df = dfs1['AADT_Unrounded']

# get all columns that start with 'AADT'
aadt_cols = [col for col in df.columns if col.startswith('AADT')]

# define the id columns that you want to keep
id_vars = ['Route', 'Beg MP', 'End MP']

# melt the DataFrame
dfUnder2021 = df.melt(id_vars=id_vars, value_vars=aadt_cols, var_name='YEAR', value_name='AADT')

# replace 'AADT' in the 'YEAR' column and convert to integer
dfUnder2021['YEAR'] = dfUnder2021['YEAR'].str.replace('AADT', '').astype(int)

# rename columns to match pre-2021 data
dfUnder2021.rename(columns={'Route':'ROUTE_ID','Beg MP':'FROM_MEASURE','End MP':'TO_MEASURE'}, inplace=True)

display(dfUnder2021)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.000,46.038,2020,430
1,0006PM,46.038,77.556,2020,385
2,0006PM,77.556,82.897,2020,552
3,0006PM,82.897,83.911,2020,2061
4,0006PM,83.911,87.694,2020,3409
...,...,...,...,...,...
181475,3468PM,0.000,5.125,1981,0
181476,3469PM,0.000,6.930,1981,0
181477,3470PM,0.000,1.039,1981,0
181478,3478PM,0.000,2.040,1981,0


In [44]:
# combine 2022, 2021 and under 2021
dfAADT = pd.concat([dfUnder2021,df2021,df2022])

dfAADT['FROM_MEASURE'] = dfAADT['FROM_MEASURE'].round(2)
dfAADT['TO_MEASURE'] = dfAADT['TO_MEASURE'].round(2)

# melt for easier joining
dfAADT = pd.pivot_table(dfAADT,index=('ROUTE_ID','FROM_MEASURE','TO_MEASURE'),columns='YEAR',values='AADT')

dfAADT.reset_index(inplace=True)
dfAADT.fillna('0',inplace=True)

# Filter rows with 'ROUTE_ID' containing 'PM' - Positive Direction & Mainline?
dfAADT_filtered = dfAADT[dfAADT['ROUTE_ID'].str.contains('PM')].copy()

# Trim 'ROUTE_ID' down to the first four characters in order to match with SEGID
dfAADT_filtered['ROUTE_ID'] = dfAADT_filtered['ROUTE_ID'].str[:4]

dfAADT_filtered

YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0006,0.00,46.04,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
1,0006,46.04,77.56,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0
2,0006,77.56,82.90,520.0,535.0,630.0,780.0,790.0,790.0,800.0,...,420.0,450.0,480.0,512.0,525.0,529.0,533.0,552.0,609.0,566.0
3,0006,82.90,83.91,0.0,1350.0,1450.0,1650.0,1675.0,1600.0,1525.0,...,1575.0,1675.0,1800.0,1913.0,1961.0,1975.0,1991.0,2061.0,2272.0,2113.0
4,0006,83.91,87.69,0.0,2650.0,2750.0,3000.0,3050.0,2920.0,2790.0,...,3340.0,3495.0,3660.0,3846.0,3468.0,3496.0,3573.0,3409.0,3852.0,3910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4807,3468,0.00,5.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2085.0,2120.0,2245.0,2354.0,2452.0,2550.0,2581.0,2578.0,2764.0,2733.0
4808,3469,0.00,6.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,505.0,515.0,545.0,572.0,333.0,346.0,350.0,350.0,375.0,371.0
4809,3470,0.00,1.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1653.0,1694.0,1724.0,1540.0,1669.0,1686.0
4810,3478,0.00,2.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1090.0,1110.0,1175.0,1232.0,865.0,900.0,911.0,910.0,976.0,965.0


In [45]:
# CHECK FOR DUPLICATES

# Define the columns you want to check for duplicates
columns_to_check = ['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE']

# Use the duplicated method to find duplicates in those columns
duplicates = dfAADT_filtered.duplicated(subset=columns_to_check, keep=False)

# Filter the DataFrame to only include the duplicates
dfAADT_duplicates = dfAADT_filtered[duplicates]

display(dfAADT_duplicates)

YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022


In [46]:
# JOIN DATA BACK TO SEGMENTS FILE AND FILTER WITH MIDPOINT_MP BETWEEN BMP AND EMP

# filter by plan area
_df = gdfSegments#[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

# calculate midpoint for 
_df['MIDPOINT_MP'] = (_df['BMP'] + _df['EMP']) / 2
_df['ROUTE_ID'] = _df['SEGID'].str.split('_').str[0]

# get only data needed to join
_df = _df[['SEGID','ROUTE_ID','MIDPOINT_MP']]

# create temp dataframe to merge to segments data
_df2 = pd.DataFrame.merge(_df, dfAADT_filtered, on='ROUTE_ID')

# filter for only segments with midpoints between AADT FROM_MEASURE and TO_MEASURE
_df2 = _df2[_df2['MIDPOINT_MP'].between(_df2['FROM_MEASURE'],_df2['TO_MEASURE'])]

# give me only columns with years
cols_to_keep = [col for col in _df2.columns if str(col).isdigit() and 1981 <= int(col) <= 3000]

# filter by only segid and columns with years
_df2 = _df2[['SEGID']+cols_to_keep]

# melt to make table long
_df2 = _df2.melt(id_vars='SEGID',var_name="YEAR",value_name='AADT')

# convert AADT to int
_df2['AADT'] = _df2['AADT'].astype(int)

# add source
_df2['SOURCE'] = fnExcelAADTHistory.split('/')[-1]

# filter out an zero data
_df2 = _df2[_df2['AADT']>0]

dfAadtFromHistoric = _df2

dfAadtFromHistoric

,SEGID,YEAR,AADT,SOURCE
0,0006_000.0,1981,325,AADTHistory.xlsx
1,0006_000.7,1981,325,AADTHistory.xlsx
2,0006_016.0,1981,325,AADTHistory.xlsx
3,0006_046.0,1981,325,AADTHistory.xlsx
4,0006_060.2,1981,325,AADTHistory.xlsx
...,...,...,...,...
315121,3468_000.0,2022,2733,AADTHistory.xlsx
315122,3469_000.0,2022,371,AADTHistory.xlsx
315123,3470_000.0,2022,1686,AADTHistory.xlsx
315124,3478_000.0,2022,965,AADTHistory.xlsx


# Wrap up and export

In [47]:
dfAadt = pd.concat([dfAadtFromSegments,dfAadtFromHistoric])
dfAadt = dfAadt.sort_values(by=['SEGID','SOURCE','YEAR'])
dfAadt = dfAadt.drop_duplicates()
dfAadt

,SEGID,YEAR,AADT,SOURCE
0,0006_000.0,1981,325,AADTHistory.xlsx
7503,0006_000.0,1982,335,AADTHistory.xlsx
15006,0006_000.0,1983,430,AADTHistory.xlsx
22509,0006_000.0,1984,580,AADTHistory.xlsx
30012,0006_000.0,1985,585,AADTHistory.xlsx
...,...,...,...,...
285113,3483_000.0,2018,1030,AADTHistory.xlsx
292616,3483_000.0,2019,1042,AADTHistory.xlsx
300119,3483_000.0,2020,1041,AADTHistory.xlsx
307622,3483_000.0,2021,1116,AADTHistory.xlsx


In [48]:
# CHECK NON-NUMERIC SEGIDs!!
dfAadt['SEGID'][dfAadt['SEGID'].str.match(r'[^\d]')].drop_duplicates()


Series([], Name: SEGID, dtype: object)

In [49]:
dfAadt.rename(columns={'AADT':'obsAADT'})

# export AADT
dfAadt.to_csv('intermediate/aadt.csv',index=False)

# export AADT sources
dfAadt[['SOURCE']].drop_duplicates().to_csv('intermediate/aadt-sources.csv',index=False)

## export Forecasts
#dfForecastsFromSegments.to_csv('intermediate/previous-forecasts.csv',index=False)

## export AADT sources
#dfForecastsFromSegments[['SOURCE']].drop_duplicates().to_csv('intermediate/previous-forecasts-sources.csv',index=False)

In [50]:
dfAadt[dfAadt['SEGID']=='2330_005.2']

,SEGID,YEAR,AADT,SOURCE
170494,2330_005.2,2003,6687,AADTHistory.xlsx
177997,2330_005.2,2004,3925,AADTHistory.xlsx
185500,2330_005.2,2005,3915,AADTHistory.xlsx
193003,2330_005.2,2006,4090,AADTHistory.xlsx
200506,2330_005.2,2007,4145,AADTHistory.xlsx
208009,2330_005.2,2008,3945,AADTHistory.xlsx
215512,2330_005.2,2009,3920,AADTHistory.xlsx
223015,2330_005.2,2010,3905,AADTHistory.xlsx
230518,2330_005.2,2011,3890,AADTHistory.xlsx
238021,2330_005.2,2012,3815,AADTHistory.xlsx


# create shapefile

In [51]:
#_df = dfAadt
#_df['YEARCOL'] = "AADT" + _df['YEAR'].astype(str)
#_df['AADT'] = _df['AADT'].astype(int)
#_df2 = _df.groupby(['SEGID','YEARCOL'],as_index=False).agg(cntRows=('YEAR','count'))
#_df2 = _df.pivot(index="SEGID",columns=('YEARCOL'), values='AADT')
#
#_df2.fillna(0,inplace=True)
#
#_df3 = pd.DataFrame.merge(gdfSegments[['SEGID','geometry']],_df2, on="SEGID", how="left")
#_df3.fillna(0,inplace=True)
#
#
## Identify numeric columns
#numeric_cols = _df3.select_dtypes(include=['float64']).columns
#
## Convert numeric columns to integer type
#for col in numeric_cols:
#    _df3[col] = _df3[col].astype(int)
#
#from datetime import datetime
#
## Generate a filename with the current timestamp in the specified folder
#timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#
#_df3.to_file('results/Segments_State_' + timestamp + '_Draft_AADTOnly.shp')